In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import nbimporter
from function import combine_data
import time

In [2]:
# We see that there were a few pages that were not properly scraped from HGTV. Here we check what seasons are missing.

og_scraped_df = pd.read_csv('Resources/House_Hunters_International_Seasons_Grouped.csv', index_col = 0)
missing_seasons = [ x for x in  np.arange(1,og_scraped_df.iloc[-1,0]) if x not in og_scraped_df['Season'].tolist() ]
missing_seasons

[1, 9, 32, 50, 63, 96, 100, 108, 117, 122, 124, 129, 152]

In [3]:
# While reviewing the data, I also noticed a few seasons only had 1 episode, so we will want to rescrape them too. The last 
# several seasons correctly only have 1 episode, so we exclude them.

missing_seasons = missing_seasons + og_scraped_df.loc[og_scraped_df['Episode'] < 3, :].iloc[:-6,:]['Season'].tolist()
missing_seasons

[1, 9, 32, 50, 63, 96, 100, 108, 117, 122, 124, 129, 152, 128, 145, 150]

In [4]:
# It appears the construction of the seasons' URLs changed over time for some reason. As such, we will need to filter the 
# seasons by format and concatenate several lists together. Season 1 actually had 2 seperate pages (1a and 100).

rescrape_urls = [ f'https://www.hgtv.com/shows/house-hunters-international/episodes/{x}a' for x in missing_seasons if x == 1 ]\
    + [ f'https://www.hgtv.com/shows/house-hunters-international/episodes/{x}00' for x in missing_seasons if x < 145 ]\
    + [ f'https://www.hgtv.com/shows/house-hunters-international/episodes/{x}' for x in missing_seasons if x > 144 ]
rescrape_urls

['https://www.hgtv.com/shows/house-hunters-international/episodes/1a',
 'https://www.hgtv.com/shows/house-hunters-international/episodes/100',
 'https://www.hgtv.com/shows/house-hunters-international/episodes/900',
 'https://www.hgtv.com/shows/house-hunters-international/episodes/3200',
 'https://www.hgtv.com/shows/house-hunters-international/episodes/5000',
 'https://www.hgtv.com/shows/house-hunters-international/episodes/6300',
 'https://www.hgtv.com/shows/house-hunters-international/episodes/9600',
 'https://www.hgtv.com/shows/house-hunters-international/episodes/10000',
 'https://www.hgtv.com/shows/house-hunters-international/episodes/10800',
 'https://www.hgtv.com/shows/house-hunters-international/episodes/11700',
 'https://www.hgtv.com/shows/house-hunters-international/episodes/12200',
 'https://www.hgtv.com/shows/house-hunters-international/episodes/12400',
 'https://www.hgtv.com/shows/house-hunters-international/episodes/12900',
 'https://www.hgtv.com/shows/house-hunters-intern

In [6]:
# Here we rescrape the several pages that were determined to have errors.

number_list = [] 
url_list = []
title_list = []
description_list = []

for url in rescrape_urls:
    executable_path = {'executable_path': ChromeDriverManager().install()}
    browser = Browser('chrome', **executable_path, headless=False)
    
    browser.visit(url)
    html = browser.html
    soup = bs(html, 'html.parser')

    episodes = soup.find('div', id = 'mod-episode-list-1')

    episode_number = episodes.find_all('span', class_='m-EpisodeCard__a-AssetInfo')
    episode_url = episodes.find_all('a')
    episode_description = episodes.find_all('p', class_='m-EpisodeCard__a-Description')

    numbers, urls, titles, descriptions = combine_data(episode_number, episode_url, episode_description)

    number_list = number_list + numbers
    url_list = url_list + urls
    title_list = title_list + titles
    description_list = description_list + descriptions
    
    browser.quit()
    
    time.sleep(1)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280


[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/87.0.4280.88/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\Dan\.wdm\drivers\chromedriver\win32\87.0.4280.88]
[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\Dan\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\Dan\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\Dan\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\Dan\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


...

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\Dan\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\Dan\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\Dan\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\Dan\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\Dan\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


In [7]:
# We build a dictionary and then a Data Frame from the 4 lists returned by our web scraper.

episode_dict = {}
episode_dict['Number'] = number_list
episode_dict['Title'] = title_list
episode_dict['Description'] = description_list
episode_dict['Link'] = url_list

episodes_df = pd.DataFrame(episode_dict)

In [8]:
# Some data cleaning to remove duplicates and seperate the 'Number' column into 'Season' and 'Episode'.

episodes_df.drop_duplicates(subset = ['Title'])

episodes_df[['Season','Episode']] = episodes_df['Number'].str.split(', ', expand = True)

episodes_df['Season'] = episodes_df['Season'].map(lambda x:x.replace('Season ', ''))
episodes_df['Episode'] = episodes_df['Episode'].map(lambda x:x.replace('Episode ', '')).astype(int)
episodes_df

,Number,Title,Description,Link,Season,Episode
0,"Season 1, Episode 1",Costa Rica Vacation Home,Mark and Carolle Cotter love living in Northri...,www.hgtv.com/shows/house-hunters-international...,1,1
1,"Season 1, Episode 2",Dominican Vacation Home,"Throughout their fifteen years of marriage, St...",www.hgtv.com/shows/house-hunters-international...,1,2
2,"Season 1, Episode 3",Vacation Home in Spain,Lesley and Clive have been together for more t...,www.hgtv.com/shows/house-hunters-international...,1,3
3,"Season 1, Episode 4",Vacation Home in Italy,"Peter and Beverly Moule live in Goff's Oak, ju...",www.hgtv.com/shows/house-hunters-international...,1,4
4,"Season 1, Episode 5",Vacation Home in Paris,New Yorkers Jeff Ballinger and Mary Schiller r...,www.hgtv.com/shows/house-hunters-international...,1,5
...,...,...,...,...,...,...
205,"Season 150, Episode 9",Luxury vs. Frugality in Hoi An,"A travel-savvy couple moves to Hoi An, Vietnam...",www.hgtv.com/shows/house-hunters-international...,150,9
206,"Season 150, Episode 10",Engineering a Life Without Carpet in Auckland,"A single man relocates to Auckland, New Zealan...",www.hgtv.com/shows/house-hunters-international...,150,10
207,"Season 150, Episode 11","Waking from Hibernation in Sosua, Dominican Re...",A family is tired of cold Toronto winters and ...,www.hgtv.com/shows/house-hunters-international...,150,11
208,"Season 150, Episode 12",French Dreams in Salies-de-Bearn,"After unexpected tragedy strikes, a romance wr...",www.hgtv.com/shows/house-hunters-international...,150,12


In [9]:
episodes_df.to_csv('Resources/House_Hunters_International_Rescraped.csv')